PART 1: SETUP AND IMPORT CSV FILES


In [1]:
# Step 1: Import Python libraries
import pandas as pd
import os

In [3]:
# Step 2: Set the path to your folder
folder_path = r'/Users/tanmaypatel/Desktop/data'  # ← Update this with your actual folder path

# Step 3: Load a few CSV files
df_patients = pd.read_csv(os.path.join(folder_path, 'patient_info.csv'))
df_symptoms = pd.read_csv(os.path.join(folder_path, 'registered_factors.csv'))
df_sdoh = pd.read_csv(os.path.join(folder_path, 'registered_sdoh.csv'))
df_goals = pd.read_csv(os.path.join(folder_path, 'goals.csv'))
df_incident = pd.read_csv(os.path.join(folder_path, 'tbi_incident.csv'))

In [5]:
# Preiew Each file

print("Patient Info")
print(df_patients.head())

print("Symptoms at Registration")
print(df_symptoms.head())

print("Social Determinants of Health")
print(df_sdoh.head())

print("Patient Goals")
print(df_goals.head())

print("TBI Incidents")
print(df_incident.head())


Patient Info
                             patient_id first_name last_name date_of_birth  \
0  5c96ba1a-8b2d-49bc-8e8e-b07761948286     Robin     Lopez     1973-05-23   
1  eda39327-b38f-41de-a46a-8782787369b7     Justin     Macks    1991-07-10   
2  6a7f7ce9-f63e-4651-b941-a25ce116de74     sandra   kinstle    1955-02-15   
3  096d402a-5fcd-41dd-b59b-b3089cf06742       Jing        Gu    1971-12-24   
4  1715cc3c-bea6-4d7e-aad3-b2f96338ba60    Bootsie   Brenner    1958-01-28   

   gender patient_type external_id    patient_sub_type  
0  female    caregiver         NaN                 NaN  
1    male    caregiver         NaN                 NaN  
2  female    caregiver         NaN                 NaN  
3    male       Stroke         NaN  PreviousStrokeUser  
4  female       Stroke         NaN  PreviousStrokeUser  
Symptoms at Registration
     id                         user_input_id symptom_date  had_symptom  \
0  1146  fdd46153-1591-47e6-bb78-3ece1f7b7f9b   2021-06-17         True   
1

PART 2: CLEAN AND PREPARE YOUR DATA

In [8]:
# PART 2.1: Clean the Patient Info Table

# Rename columns for consistency
df_patients = df_patients.rename(columns={
    'patient_id': 'PatientID',
    'date_of_birth': 'DOB',
    'gender': 'Gender',
    'patient_type': 'PatientType',
    'patient_sub_type': 'PatientSubType'
})

# Convert date_of_birth to datetime format
df_patients['DOB'] = pd.to_datetime(df_patients['DOB'], errors='coerce')

# Create an Age column from DOB
df_patients['Age'] = 2024 - df_patients['DOB'].dt.year

# Keep only relevant columns
df_patients_clean = df_patients[['PatientID', 'Gender', 'Age', 'PatientType', 'PatientSubType']]


In [10]:
# PART 2.2: Clean the Registered Symptoms Table

df_symptoms = df_symptoms.rename(columns={
    'patient_about_id': 'PatientID',
    'factor': 'Symptom',
    'category': 'SymptomCategory'
})

df_symptoms_clean = df_symptoms[['PatientID', 'Symptom', 'SymptomCategory']]


In [12]:
# PART 2.3: Clean the Registered SDOH Table

df_sdoh = df_sdoh.rename(columns={
    'patient_about_id': 'PatientID',
    'factor': 'Symptom',
    'category': 'SymptomCategory'
})

df_sdoh_clean = df_sdoh[['PatientID', 'Symptom', 'SymptomCategory']]


In [14]:
# PART 2.4: Combine Medical and SDOH Symptoms into One Table

df_all_symptoms = pd.concat([df_symptoms_clean, df_sdoh_clean], ignore_index=True)


In [16]:
# PART 2.5: Clean the Goals Table (Optional)

df_goals = df_goals.dropna(subset=['first_goal', 'second_goal'], how='all')


PART 3: Merge, Handle Missing Data, and Start Analysis

In [19]:
# PART 3.1: Merge Patient Info with Symptom Data

# Merge patient demographics with their symptoms
df_merged = pd.merge(df_all_symptoms, df_patients_clean, on='PatientID', how='left')

print(df_merged.head())


                              PatientID                     Symptom  \
0  1eaca131-cd7f-4954-a3ce-262cacfa8390                       Angry   
1  a788c975-548b-45f7-8f83-36f51c1edd4a  Can't find the right words   
2  6508f277-d031-4fbe-95a6-165e8d61e802    Brain Fog, Lack of Focus   
3  6508f277-d031-4fbe-95a6-165e8d61e802             Extreme Yawning   
4  6508f277-d031-4fbe-95a6-165e8d61e802                     Shaking   

  SymptomCategory  Gender  Age     PatientType PatientSubType  
0         medical  female   72  TBI/Concussion     tbiPatient  
1         medical  female   34  TBI/Concussion     tbiPatient  
2         medical  female   36  TBI/Concussion     tbiPatient  
3         medical  female   36  TBI/Concussion     tbiPatient  
4         medical  female   36  TBI/Concussion     tbiPatient  


In [21]:
# PART 3.2: Check for Missing Values

print(df_merged.isnull().sum())

PatientID            0
Symptom            122
SymptomCategory      0
Gender               0
Age                  0
PatientType          4
PatientSubType     137
dtype: int64


In [23]:
# Drop rows where Gender or Age is missing (optional)
df_merged = df_merged.dropna(subset=['Gender', 'Age'])

In [25]:
# PART 3.3: Analyze Symptoms by Gender

symptom_by_gender = df_merged.groupby(['Gender', 'SymptomCategory']).size().reset_index(name='Count')
print(symptom_by_gender.head())


   Gender SymptomCategory  Count
0   Other            SDOH     23
1   Other         medical     21
2  female            SDOH   3767
3  female         medical   5933
4    male            SDOH   1613


In [27]:
# PART 3.4: Analyze Symptoms by Age Group

# Define age groups
bins = [0, 18, 30, 45, 60, 100]
labels = ['<18', '18-30', '31-45', '46-60', '60+']
df_merged['AgeGroup'] = pd.cut(df_merged['Age'], bins=bins, labels=labels)

#symptom_by_age = df_merged.groupby(['AgeGroup', 'SymptomCategory']).size().reset_index(name='Count')
symptom_by_age = df_merged.groupby(['AgeGroup', 'SymptomCategory'], observed=False).size().reset_index(name='Count')
print(symptom_by_age.head())

  AgeGroup SymptomCategory  Count
0      <18            SDOH    181
1      <18         medical    254
2    18-30            SDOH    624
3    18-30         medical   1344
4    31-45            SDOH   1932


In [29]:
df_merged.to_csv('Cleaned_Symptom_Patient_Data.csv', index=False)


PART 4: Enrich the Dataset with Injury Details and Prepare Final Dashboard File

In [32]:
# PART 4.1: Load and Inspect Injury Data Files

# Load injury-related files
df_tbi_incident = pd.read_csv(os.path.join(folder_path, 'tbi_incident.csv'))
df_head_hit = pd.read_csv(os.path.join(folder_path, 'incident_head_hit_location.csv'))
df_immediate_symptoms = pd.read_csv(os.path.join(folder_path, 'immediate_symptoms_resulting.csv'))

In [34]:
# PART 4.2: Clean TBI Incident File

df_tbi_incident = df_tbi_incident.rename(columns={
    'patient_id': 'PatientID',
    'tbi_incident_date': 'InjuryDate',
    'describe_event': 'InjuryDescription',
    'injury_from': 'InjuryCause'
})

# Keep relevant columns
df_tbi_incident_clean = df_tbi_incident[['PatientID', 'InjuryDate', 'InjuryDescription', 'InjuryCause']]


In [36]:
# PART 4.3: Clean Head Hit Location File

df_head_hit = df_head_hit.rename(columns={
    'patient_id': 'PatientID',
    'head_hit_location': 'HeadHitLocation'
})

df_head_hit_clean = df_head_hit[['PatientID', 'HeadHitLocation']]


In [38]:
df_head_hit_grouped = (
    df_head_hit_clean
    .dropna(subset=['HeadHitLocation'])  # remove missing values
    .groupby('PatientID')['HeadHitLocation']
    .apply(lambda x: ', '.join(x.dropna().astype(str).unique()))
    .reset_index()
)


In [40]:
# PART 4.4: Clean Immediate Symptoms File

df_immediate_symptoms = df_immediate_symptoms.rename(columns={
    'patient_id': 'PatientID',
    'immediate_symptoms_resulting': 'ImmediateSymptom'
})

df_immediate_grouped = (
    df_immediate_symptoms
    .dropna(subset=['ImmediateSymptom'])  # remove missing values
    .groupby('PatientID')['ImmediateSymptom']
    .apply(lambda x: ', '.join(x.dropna().astype(str).unique()))
    .reset_index()
)

In [42]:
# PART 4.5: Merge Injury Details into Main Dataset

# Merge injury description
df_merged = pd.merge(df_merged, df_tbi_incident_clean, on='PatientID', how='left')

# Merge head-hit location
df_merged = pd.merge(df_merged, df_head_hit_grouped, on='PatientID', how='left')

# Merge immediate symptoms
df_merged = pd.merge(df_merged, df_immediate_grouped, on='PatientID', how='left')

In [44]:
# PART 4.6: Save Final Dashboard Dataset

df_merged.to_csv('Final_Dashboard_Ready_Data.csv', index=False)


In [47]:
# Step 1: Load patient_therapies.csv
therapy_file_path = os.path.join(folder_path, 'patient_therapies.csv')  # Make sure folder_path is already defined
df_therapy = pd.read_csv(therapy_file_path)

# Step 2: Keep only necessary columns
df_therapy_clean = df_therapy[['patient_id', 'therapies', 'category']].copy()
df_therapy_clean = df_therapy_clean.rename(columns={
    'patient_id': 'PatientID',
    'therapies': 'TherapyReceived',
    'category': 'TherapyCategory'
})

# Step 3: Merge into your final cleaned dataset
df_final_with_therapy = pd.merge(df_merged, df_therapy_clean, on='PatientID', how='left')

# Step 4: Save merged dataset (optional)
df_final_with_therapy.to_csv('Final_Dataset.csv', index=False)

# Step 5: Preview the result
df_final_with_therapy.head()


,PatientID,Symptom,SymptomCategory,Gender,Age,PatientType,PatientSubType,AgeGroup,InjuryDate,InjuryDescription,InjuryCause,HeadHitLocation,ImmediateSymptom,TherapyReceived,TherapyCategory
0,1eaca131-cd7f-4954-a3ce-262cacfa8390,Angry,medical,female,72,TBI/Concussion,tbiPatient,60+,6/18/21,Middle of the night fall down stairs,Fall,Left Side of Head,"Loss of Consciousness, Confusion, Memory Loss",NaN,NaN
1,1eaca131-cd7f-4954-a3ce-262cacfa8390,Angry,medical,female,72,TBI/Concussion,tbiPatient,60+,6/18/21,Middle of the night fall down stairs,Fall,Left Side of Head,"Loss of Consciousness, Confusion, Memory Loss",Physical Therapy,Applied
2,1eaca131-cd7f-4954-a3ce-262cacfa8390,Angry,medical,female,72,TBI/Concussion,tbiPatient,60+,6/18/21,Middle of the night fall down stairs,Fall,Left Side of Head,"Loss of Consciousness, Confusion, Memory Loss",NaN,NaN
3,1eaca131-cd7f-4954-a3ce-262cacfa8390,Angry,medical,female,72,TBI/Concussion,tbiPatient,60+,6/18/21,Middle of the night fall down stairs,Fall,Left Side of Head,"Loss of Consciousness, Confusion, Memory Loss",Developmental Vision Therapy,Vision
4,1eaca131-cd7f-4954-a3ce-262cacfa8390,Angry,medical,female,72,TBI/Concussion,tbiPatient,60+,6/18/21,Middle of the night fall down stairs,Fall,Left Side of Head,"Loss of Consciousness, Confusion, Memory Loss",NaN,NaN


In [49]:
# For your final dataset
print("Shape of the dataset (rows, columns):", df_final_with_therapy.shape)

Shape of the dataset (rows, columns): (103338, 15)


In [52]:
import pandas as pd

# Load new summary file (replace with actual path if different)
new_summary_df = pd.read_csv('/Users/tanmaypatel/Desktop/data/IISS/pofp_df.csv')

# Step 1: Ensure consistent naming for merge key
new_summary_df.rename(columns={'patient_id': 'PatientID'}, inplace=True)

# Step 2: Remove unwanted columns if needed (e.g., nested tuples or debug fields)
# For now, we keep them; user may later clean or flatten those fields manually

# Step 3: Merge on PatientID using a left join
merged_df = df_final_with_therapy.merge(new_summary_df, on='PatientID', how='left')

# Step 4: Show shape and sample to confirm
merged_shape = merged_df.shape
merged_sample = merged_df.sample(5, random_state=42)

merged_shape, merged_sample.head()


((103338, 123),
                                   PatientID  \
 74158  55d2003a-8664-488f-b075-5902e950ab6f   
 26637  1b0755ec-e30c-4477-8e62-c1330829316a   
 36729  4a941b7f-fcb0-4046-bc28-660967cf48e1   
 69246  a05344cd-6fed-405b-8135-e3485d8af36e   
 38420  1b0755ec-e30c-4477-8e62-c1330829316a   
 
                                                  Symptom SymptomCategory  \
 74158                                            Friends            SDOH   
 26637                                  Chronically Itchy         medical   
 36729  Line skipping: when reading text (paper or com...         medical   
 69246                                     Food poisoning            SDOH   
 38420                       Trouble with self-discipline         medical   
 
        Gender  Age     PatientType                 PatientSubType AgeGroup  \
 74158    male   42  TBI/Concussion                      Acute TBI    31-45   
 26637    male   50  TBI/Concussion  Post-Concussion Symptom (PCS)    46

In [56]:
# Save merged version to new file
merged_df.to_csv('Final_Data.csv', index=False)


In [60]:
import pandas as pd

# Load the final dataset
df = pd.read_csv("Final_Data.csv")

# Step 1: Define common vision-related symptoms
vision_symptoms = [
    "Blurred Vision",
    "Light Sensitivity",
    "Visual Overstimulation",
    "Eyes", "Vision", "Eye strain"
]

# Step 2: Filter for vision-symptom patients
df_vision_symptoms = df[df['Symptom'].str.contains('|'.join(vision_symptoms), case=False, na=False)]

# Step 3: Tag whether they received any Vision Therapy
df_vision_symptoms['Received_Vision_Therapy'] = df_vision_symptoms['TherapyCategory'].str.contains('Vision', case=False, na=False)

# Step 4: Calculate key metrics
total_vision_patients = df_vision_symptoms['PatientID'].nunique()
vision_therapy_patients = df_vision_symptoms[df_vision_symptoms['Received_Vision_Therapy']]['PatientID'].nunique()
no_therapy_patients = total_vision_patients - vision_therapy_patients

percent_with_therapy = (vision_therapy_patients / total_vision_patients) * 100 if total_vision_patients else 0
percent_without_therapy = (no_therapy_patients / total_vision_patients) * 100 if total_vision_patients else 0

# To view the data table in Jupyter
df_vision_symptoms.head(10)  # or however many rows you want to preview

{
    "Total Patients with Vision Symptoms": total_vision_patients,
    "Received Vision Therapy": vision_therapy_patients,
    "Did Not Receive Vision Therapy": no_therapy_patients,
    "% Received Therapy": round(percent_with_therapy, 2),
    "% Did Not Receive Therapy": round(percent_without_therapy, 2)
}


/var/folders/nz/kt4hj4sn7m59n2m_fnl42m4m0000gn/T/ipykernel_2460/594734878.py:4: DtypeWarning: Columns (16,17,18,20,21,22,23,59,60,61,62,63,64,65,66,67,68,70,71,72,73,77,78,79,80,84,85,86,87,91,93,94,98,99,100,101,105,106,107,108,111,113,115,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Final_Data.csv")
/var/folders/nz/kt4hj4sn7m59n2m_fnl42m4m0000gn/T/ipykernel_2460/594734878.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vision_symptoms['Received_Vision_Therapy'] = df_vision_symptoms['TherapyCategory'].str.contains('Vision', case=False, na=False)


{'Total Patients with Vision Symptoms': 236,
 'Received Vision Therapy': 46,
 'Did Not Receive Vision Therapy': 190,
 '% Received Therapy': 19.49,
 '% Did Not Receive Therapy': 80.51}

In [66]:
# Load your final dataset first
df_final = pd.read_csv("Final_Data.csv", low_memory=False)

# Grouping symptom categories and therapy mappings
category_map = {
    'vision': 'Vision',
    'emotional': 'Mental Health',
    'cognitive': 'Mental Health',  # Depending on therapy options, might include Occupational Therapy
    'physical': 'Applied',         # e.g., Physical Therapy
    'sleep': 'Mental Health',      # Some overlap with sleep therapies
    'speech': 'Applied'            # Speech Language Therapy often falls here
}


# Clean up symptom category values
df_final['SymptomCategory'] = df_final['SymptomCategory'].astype(str).str.strip().str.lower()
df_final['TherapyCategory'] = df_final['TherapyCategory'].astype(str).str.strip().str.lower()


# Create a new column: Is Therapy Matched?
def check_match(row):
    expected = category_map.get(row['SymptomCategory'])
    return expected.lower() in row['TherapyCategory'].lower() if expected else False

df_final['TherapyMatch'] = df_final.apply(check_match, axis=1)

# Group by Symptom Category
results = []
for category in category_map.keys():
    df_cat = df_final[df_final['SymptomCategory'] == category]
    total_patients = df_cat['PatientID'].nunique()
    matched = df_cat[df_cat['TherapyMatch'] == True]['PatientID'].nunique()
    unmatched = total_patients - matched
    results.append({
        'SymptomCategory': category.title(),
        'Total Patients': total_patients,
        'Received Matching Therapy': matched,
        'Did Not Receive Matching Therapy': unmatched,
        '% Received Matching Therapy': round((matched / total_patients * 100) if total_patients else 0, 2),
        '% Did Not Receive Therapy': round((unmatched / total_patients * 100) if total_patients else 0, 2)
    })

# Final DataFrame
import pandas as pd
df_therapy_match_summary = pd.DataFrame(results)
df_therapy_match_summary


,SymptomCategory,Total Patients,Received Matching Therapy,Did Not Receive Matching Therapy,% Received Matching Therapy,% Did Not Receive Therapy
0,Vision,0,0,0,0,0
1,Emotional,0,0,0,0,0
2,Cognitive,0,0,0,0,0
3,Physical,0,0,0,0,0
4,Sleep,0,0,0,0,0
5,Speech,0,0,0,0,0


In [78]:
import pandas as pd
import ast

# Load your data
df_final = pd.read_csv("Final_Data.csv", low_memory=False)

# Step 1: Clean and convert the 'subcategory' string to a list
def parse_subcategory(val):
    try:
        return ast.literal_eval(val)
    except:
        return []

df_final['subcategory_clean'] = df_final['subcategory'].apply(parse_subcategory)

# Step 2: Explode into multiple rows for each subcategory
df_exploded = df_final.explode('subcategory_clean')

# Step 3: Normalize text (lowercase and strip)
df_exploded['subcategory_clean'] = df_exploded['subcategory_clean'].str.strip().str.lower()
df_exploded['TherapyCategory'] = df_exploded['TherapyCategory'].fillna('').str.strip().str.lower()

# Step 4: Map subcategory to expected therapy category
subcategory_therapy_map = {
    'vision': 'vision',
    'speech': 'applied',
    'emotional': 'mental health',
    'cognitive': 'mental health',
    'physical': 'applied',
    'sleep': 'medical'
}

# Step 5: Create a column to mark therapy match
df_exploded['ExpectedTherapy'] = df_exploded['subcategory_clean'].map(subcategory_therapy_map)
df_exploded['TherapyMatched'] = df_exploded.apply(
    lambda row: 1 if pd.notna(row['ExpectedTherapy']) and row['ExpectedTherapy'] in row['TherapyCategory'] else 0,
    axis=1
)


# Step 6: Group and calculate stats
df_summary = (
    df_exploded.groupby('subcategory_clean')
    .agg(
        TotalPatients=('PatientID', 'nunique'),
        PatientsWithMatchingTherapy=('TherapyMatched', 'sum')
    )
    .reset_index()
)

# Step 7: Final calculation
df_summary['PatientsWithoutTherapy'] = df_summary['TotalPatients'] - df_summary['PatientsWithMatchingTherapy']
df_summary['% Received Matching Therapy'] = (df_summary['PatientsWithMatchingTherapy'] / df_summary['TotalPatients'] * 100).round(2)
df_summary['% Did Not Receive Therapy'] = (df_summary['PatientsWithoutTherapy'] / df_summary['TotalPatients'] * 100).round(2)

print(df_summary)


  subcategory_clean  TotalPatients  PatientsWithMatchingTherapy  \
0         cognitive             36                         1450   
1         emotional             26                         1465   
2          physical             25                         1297   
3             sleep             26                            0   
4            speech             21                          893   
5            vision             24                           86   

   PatientsWithoutTherapy  % Received Matching Therapy  \
0                   -1414                      4027.78   
1                   -1439                      5634.62   
2                   -1272                      5188.00   
3                      26                         0.00   
4                    -872                      4252.38   
5                     -62                       358.33   

   % Did Not Receive Therapy  
0                   -3927.78  
1                   -5534.62  
2                   -5088.00  

In [80]:
# Step 1: Create a new column for match flag
df_exploded['therapy_match_flag'] = df_exploded.apply(
    lambda row: 1 if pd.notna(row['ExpectedTherapy']) and row['ExpectedTherapy'] in row['TherapyCategory'] else 0,
    axis=1
)

# Step 2: Deduplicate to (PatientID + Subcategory) level
df_match_level = (
    df_exploded[['PatientID', 'subcategory_clean', 'therapy_match_flag']]
    .drop_duplicates()
    .groupby(['subcategory_clean', 'PatientID'])
    .agg(MatchFound=('therapy_match_flag', 'max'))  # max = 1 if at least one match
    .reset_index()
)

# Step 3: Group by subcategory and count match stats
df_summary = (
    df_match_level.groupby('subcategory_clean')
    .agg(
        TotalPatients=('PatientID', 'count'),
        PatientsWithMatchingTherapy=('MatchFound', 'sum')
    )
    .reset_index()
)

# Step 4: Final calculations
df_summary['PatientsWithoutTherapy'] = df_summary['TotalPatients'] - df_summary['PatientsWithMatchingTherapy']
df_summary['% Received Matching Therapy'] = (df_summary['PatientsWithMatchingTherapy'] / df_summary['TotalPatients'] * 100).round(2)
df_summary['% Did Not Receive Therapy'] = (df_summary['PatientsWithoutTherapy'] / df_summary['TotalPatients'] * 100).round(2)

# Save or view
print(df_summary)
df_summary.to_csv("Therapy_Matching_Summary_Corrected.csv", index=False)


  subcategory_clean  TotalPatients  PatientsWithMatchingTherapy  \
0         cognitive             36                            9   
1         emotional             26                            8   
2          physical             25                            9   
3             sleep             26                            0   
4            speech             21                            5   
5            vision             24                            2   

   PatientsWithoutTherapy  % Received Matching Therapy  \
0                      27                        25.00   
1                      18                        30.77   
2                      16                        36.00   
3                      26                         0.00   
4                      16                        23.81   
5                      22                         8.33   

   % Did Not Receive Therapy  
0                      75.00  
1                      69.23  
2                      64.00  

In [82]:
import pandas as pd

# Constructing the corrected DataFrame based on user's request
df_summary = pd.DataFrame({
    'Symptom Category': ['Cognitive', 'Emotional', 'Physical', 'Sleep', 'Speech', 'Vision'],
    'Total Patients': [36, 26, 25, 26, 21, 24],
    'Received Matching Therapy': [9, 8, 9, 0, 5, 2],
    'Did Not Receive Therapy': [27, 18, 16, 26, 16, 22],
    '% Received Matching Therapy': [25.00, 30.77, 36.00, 0.00, 23.81, 8.33],
    '% Did Not Receive Therapy': [75.00, 69.23, 64.00, 100.00, 76.19, 91.67]
})

df_summary

,Symptom Category,Total Patients,Received Matching Therapy,Did Not Receive Therapy,% Received Matching Therapy,% Did Not Receive Therapy
0,Cognitive,36,9,27,25.00,75.00
1,Emotional,26,8,18,30.77,69.23
2,Physical,25,9,16,36.00,64.00
3,Sleep,26,0,26,0.00,100.00
4,Speech,21,5,16,23.81,76.19
5,Vision,24,2,22,8.33,91.67
